In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df_execution = pd.read_csv("C:\Users\Stijn\Desktop\\3de jaar\Module 1\Data mining\Project_DeathRow\execution_database.csv")
df_crime = pd.read_csv("C:\Users\Stijn\Desktop\\3de jaar\Module 1\Data mining\Project_DeathRow\Crime_Alabama.csv")

In [31]:
print df_execution.values[99][0]
print df_crime.values


05/04/1990
[['1960' '   3266740' '       6097' ..., 592.1 87.3 nan]
 ['1961' '   3302000' '       5564' ..., 569.4 76.8 nan]
 ['1962' '   3358000' '       5283' ..., 634.5 83.4 nan]
 ..., 
 [ '           Rates are the number of reported offenses per 100,000 population "         Alabama - <ul><li>2011 - because of changes in the state\'s reporting practices, figures are not comparable to previous years\' data.</li></ul>"'
  nan nan ..., nan nan nan]
 ['Sources: FBI' ' Uniform Crime Reports'
  ' prepared by the National Archive of Criminal Justice Data ' ..., nan
  nan nan]
 ['Date of download: Oct 10 2016 ' nan nan ..., nan nan nan]]
